In [ ]:
!pip install pytorch-msssim
!pip install faiss-cpu
!pip install wandb
!pip install sympy

In [ ]:
from torchvision import datasets, transforms

from torchvision.utils import save_image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from torchsummary import summary
import torch.optim as optim
import logging

import wandb
import random

import  torch, time, os, pickle
import os
import torch
import numpy as np

from os import listdir
 

import torch
import torch.nn.functional as F

from pytorch_msssim import ssim  # For SSIM metric
import numpy as np
import torch.nn as nn
import torch.optim as optim


import matplotlib.image as mpimg


In [ ]:
# Define device as cuda if available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def save_images(epoch,y_,model_type):
    os.makedirs('outputs', exist_ok=True)
    z_ = torch.rand((batch_size, z_dim))
    y_vec_ = torch.zeros((batch_size, class_num)) \
            .scatter_(1, y_.type(torch.LongTensor).unsqueeze(1), 1)
    fake_images = G(z_,y_vec_).detach().cpu()
    save_image(fake_images, f'outputs/'+model_type+str(epoch)+'.png', nrow=8, normalize=True)

In [ ]:
def generated_images():
  folder_dir = "/content/outputs/"

  for image in os.listdir(folder_dir):
      img = mpimg.imread(folder_dir+image)
      plt.imshow(img)
      plt.axis('off')  
      plt.show()

In [ ]:
class GeneratorCGAN(nn.Module):
    def __init__(self, nz=100, nc=1, input_size=32, class_num=10):
        super(GeneratorCGAN, self).__init__()
        self.nz = nz
        self.nc = nc
        self.input_size = input_size
        self.class_num = class_num

      
        self.fc = nn.Sequential(
            nn.Linear(self.nz + self.class_num, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.ReLU(),
        )
    
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, self.nc, 4, 2, 1),
            nn.Tanh(),  
        )

        self.apply(self._initialize_weights)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        

        x = self.fc(x)
        x = x.view(-1, 128, (self.input_size // 4), (self.input_size // 4))

    
        x = self.deconv(x)

        return x

    def _initialize_weights(self, m):
        if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
